In [4]:
import os
import json
import pandas as pd
from glob import glob

# Paths
details_dir = "01_raw_data/01_species_details"
output_path = "02_wrangled_data/Table05_GeneralPlantImageTable.csv"

# 获取所有物种详情 JSON 文件
detail_files = glob(os.path.join(details_dir, "plant_species_details_*.json"))

records = []
for file in detail_files:
    with open(file, "r", encoding="utf-8") as f:
        details = json.load(f)
    
    plant_id = details.get("id")
    
    # 跳过 threatened plants (ID > 3000)
    if plant_id > 3000:
        continue
    
    # 提取图片 URL
    default_image = details.get("default_image", {})
    regular_url = default_image.get("regular_url", "") if default_image else ""
    thumbnail_url = default_image.get("thumbnail", "") if default_image else ""
    
    records.append({
        "general_plant_id": plant_id,
        "regular_url_image": regular_url,
        "thumbnail_image": thumbnail_url
    })

# 创建 DataFrame 并排序
df = pd.DataFrame(records)
df = df.sort_values(by="general_plant_id").reset_index(drop=True)
df["general_plant_id"] = pd.to_numeric(df["general_plant_id"], errors="coerce").astype("Int64")

# 保存到 CSV - 使用 CRLF 行终止符以保持一致性
os.makedirs(os.path.dirname(output_path), exist_ok=True)
df.to_csv(output_path, index=False, encoding='utf-8', lineterminator='\r\n')

print(f"成功处理 {len(records)} 个植物图像记录，已保存到 {output_path}")

成功处理 593 个植物图像记录，已保存到 02_wrangled_data/Table05_GeneralPlantImageTable.csv


In [5]:
# 验证数据
print("\n前5行数据预览:")
df.head()


前5行数据预览:


,general_plant_id,regular_url_image,thumbnail_image
0,1,https://perenual.com/storage/species_image/1_a...,https://perenual.com/storage/species_image/1_a...
1,2,https://perenual.com/storage/species_image/2_a...,https://perenual.com/storage/species_image/2_a...
2,3,https://perenual.com/storage/species_image/3_a...,https://perenual.com/storage/species_image/3_a...
3,4,https://perenual.com/storage/species_image/4_a...,https://perenual.com/storage/species_image/4_a...
4,5,https://perenual.com/storage/species_image/5_a...,https://perenual.com/storage/species_image/5_a...


In [6]:
# 验证行终止符
with open(output_path, 'rb') as f:
    content = f.read(200)  # 读取前200字节
    
if b'\r\n' in content:
    print("✅ CSV 文件使用 CRLF (\\r\\n) 行终止符")
else:
    print("❌ CSV 文件未使用 CRLF 行终止符")

✅ CSV 文件使用 CRLF (\r\n) 行终止符


## 执行 LOAD DATA LOCAL INFILE 命令上传到数据库

In [7]:
import mysql.connector
from mysql.connector import Error

# 数据库连接配置
db_config = {
    'host': 'database-plantx.cqz06uycysiz.us-east-1.rds.amazonaws.com',
    'user': 'zihan',
    'password': '2002317Yzh12138.',
    'database': 'FIT5120_PlantX_Database',
    'allow_local_infile': True,
    'use_pure': True  # 使用纯Python实现
}

try:
    # 建立连接
    connection = mysql.connector.connect(**db_config)
    
    if connection.is_connected():
        print("成功连接到 MySQL 服务器")
        
        # 创建游标
        cursor = connection.cursor()
        
        # 构建 LOAD DATA LOCAL INFILE 命令
        # 注意：请将下面的文件路径替换为你实际的CSV文件路径
        load_data_query = """
        LOAD DATA LOCAL INFILE '02_wrangled_data/Table05_GeneralPlantImageTable.csv'
        INTO TABLE Table05_GeneralPlantImageTable
        CHARACTER SET utf8mb4
        FIELDS TERMINATED BY ',' 
        OPTIONALLY ENCLOSED BY '"'
        LINES TERMINATED BY '\\r\\n'
        IGNORE 1 LINES
        (   
            general_plant_id, regular_url_image, thumbnail_image
        );
        """
        
        # 执行命令
        cursor.execute(load_data_query)
        connection.commit()  # 提交事务
        
        print(f"数据导入成功！影响了 {cursor.rowcount} 行。")
        
except Error as e:
    print(f"执行过程中发生错误：{e}")
    
finally:
    # 关闭连接
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL 连接已关闭。")

成功连接到 MySQL 服务器
数据导入成功！影响了 99 行。
MySQL 连接已关闭。


## 验证导入结果

In [8]:
# 在同一个连接会话中，或者在新的连接中执行
try:
    connection = mysql.connector.connect(**db_config)
    cursor = connection.cursor()
    
    cursor.execute("SELECT COUNT(*) FROM Table05_GeneralPlantImageTable")
    row_count = cursor.fetchone()[0]
    print(f"表中现有 {row_count} 行数据")
    
    # 查看前几行数据
    cursor.execute("SELECT * FROM Table05_GeneralPlantImageTable LIMIT 5")
    rows = cursor.fetchall()
    for row in rows:
        print(row)
        
except Error as e:
    print(f"查询过程中发生错误：{e}")
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()

表中现有 593 行数据
(1, 'https://perenual.com/storage/species_image/1_abies_alba/regular/1536px-Abies_alba_SkalitC3A9.jpg', 'https://perenual.com/storage/species_image/1_abies_alba/thumbnail/1536px-Abies_alba_SkalitC3A9.jpg')
(2, 'https://perenual.com/storage/species_image/2_abies_alba_pyramidalis/regular/49255769768_df55596553_b.jpg', 'https://perenual.com/storage/species_image/2_abies_alba_pyramidalis/thumbnail/49255769768_df55596553_b.jpg')
(3, 'https://perenual.com/storage/species_image/3_abies_concolor/regular/52292935430_f4f3b22614_b.jpg', 'https://perenual.com/storage/species_image/3_abies_concolor/thumbnail/52292935430_f4f3b22614_b.jpg')
(4, 'https://perenual.com/storage/species_image/4_abies_concolor_candicans/regular/49283844888_332c9e46f2_b.jpg', 'https://perenual.com/storage/species_image/4_abies_concolor_candicans/thumbnail/49283844888_332c9e46f2_b.jpg')
(5, 'https://perenual.com/storage/species_image/5_abies_fraseri/regular/36843539702_e80fc436e0_b.jpg', 'https://perenual.com/st

## 下载并保存缩略图

In [9]:
import os
import pandas as pd
import requests
import time
from pathlib import Path

# 路径设置
csv_path = "02_wrangled_data/Table05_GeneralPlantImageTable.csv"
output_dir = "01_raw_data/05_thumbnail_image"

# 创建输出目录（如果不存在）
os.makedirs(output_dir, exist_ok=True)

# 读取 CSV 文件
df = pd.read_csv(csv_path)

# 计数器
success_count = 0
fail_count = 0

print(f"开始下载 {len(df)} 个缩略图...")

for index, row in df.iterrows():
    plant_id = row['general_plant_id']
    thumbnail_url = row['thumbnail_image']
    
    # 跳过空 URL
    if pd.isna(thumbnail_url) or not thumbnail_url:
        print(f"跳过植物 ID {plant_id}：无缩略图 URL")
        fail_count += 1
        continue
    
    # 构建文件名
    filename = f"plant_species_thumbnail_image_{plant_id}.jpg"
    filepath = os.path.join(output_dir, filename)
    
    # 如果文件已存在，跳过下载
    if os.path.exists(filepath):
        print(f"跳过植物 ID {plant_id}：文件已存在")
        success_count += 1
        continue
    
    try:
        # 直接请求图片
        response = requests.get(thumbnail_url, timeout=30)
        response.raise_for_status()
        
        # 保存图片
        with open(filepath, 'wb') as f:
            f.write(response.content)
        
        print(f"成功下载植物 ID {plant_id} 的缩略图")
        success_count += 1
        
        time.sleep(0.1)  # 短暂延迟，避免过快请求
        
    except requests.exceptions.RequestException as e:
        print(f"下载植物 ID {plant_id} 的缩略图失败: {e}")
        fail_count += 1
    except Exception as e:
        print(f"处理植物 ID {plant_id} 时发生错误: {e}")
        fail_count += 1

print(f"\n下载完成！成功: {success_count}, 失败: {fail_count}")

# 验证文件数
downloaded_files = list(Path(output_dir).glob("plant_species_thumbnail_image_*.jpg"))
print(f"缩略图文件夹中现有 {len(downloaded_files)} 个文件")

开始下载 593 个缩略图...
跳过植物 ID 1：文件已存在
跳过植物 ID 2：文件已存在
跳过植物 ID 3：文件已存在
跳过植物 ID 4：文件已存在
跳过植物 ID 5：文件已存在
跳过植物 ID 6：文件已存在
跳过植物 ID 7：文件已存在
跳过植物 ID 8：文件已存在
跳过植物 ID 9：文件已存在
跳过植物 ID 10：文件已存在
跳过植物 ID 11：文件已存在
跳过植物 ID 12：文件已存在
跳过植物 ID 13：文件已存在
跳过植物 ID 14：文件已存在
跳过植物 ID 15：文件已存在
跳过植物 ID 16：文件已存在
跳过植物 ID 17：文件已存在
跳过植物 ID 18：文件已存在
跳过植物 ID 19：文件已存在
跳过植物 ID 20：文件已存在
跳过植物 ID 21：文件已存在
跳过植物 ID 22：文件已存在
跳过植物 ID 23：文件已存在
跳过植物 ID 24：文件已存在
跳过植物 ID 25：文件已存在
跳过植物 ID 26：文件已存在
跳过植物 ID 27：文件已存在
跳过植物 ID 28：文件已存在
跳过植物 ID 29：无缩略图 URL
跳过植物 ID 30：文件已存在
跳过植物 ID 31：文件已存在
跳过植物 ID 32：文件已存在
跳过植物 ID 33：无缩略图 URL
跳过植物 ID 34：文件已存在
跳过植物 ID 35：文件已存在
跳过植物 ID 36：文件已存在
跳过植物 ID 37：文件已存在
跳过植物 ID 38：无缩略图 URL
跳过植物 ID 39：文件已存在
跳过植物 ID 40：文件已存在
跳过植物 ID 41：无缩略图 URL
跳过植物 ID 42：文件已存在
跳过植物 ID 43：文件已存在
跳过植物 ID 44：文件已存在
跳过植物 ID 45：文件已存在
跳过植物 ID 46：文件已存在
跳过植物 ID 47：文件已存在
跳过植物 ID 48：文件已存在
跳过植物 ID 49：文件已存在
跳过植物 ID 50：文件已存在
跳过植物 ID 51：文件已存在
跳过植物 ID 52：文件已存在
跳过植物 ID 53：文件已存在
跳过植物 ID 54：文件已存在
跳过植物 ID 55：文件已存在
跳过植物 ID 56：文件已存在
跳过植物 ID 57：文件已存在
跳过植物 ID 58：